# Candlestick pattern modeling

### Import Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [2]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [3]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-6)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [4]:
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

### Load Price Data

In [5]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
len(ticker_data)

6476

In [6]:
import candlestick.price_action as pa

In [7]:
data = ticker_data.copy()

In [8]:
# BACK_BARS=250
# data = data.tail(BACK_BARS)
data = data[data.index >  '2024-01-01 08:00:00']

In [9]:
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma_1w'] = data['above_ma'].rolling(250).sum()
data['total_below_ma_1w'] = data['below_ma'].rolling(250).sum()
data['trend'] = data.apply(lambda r: 'switch' if r['total_above_ma_1w'] == r['total_below_ma_1w'] else ('up' if r['total_above_ma_1w'] > r['total_below_ma_1w'] else 'down'), axis=1)
data["prev_Volume"] = data["Volume"].shift(1)

In [10]:
%%time
data = pa.pattern_modeling(data, 'reversal')

CPU times: user 4.37 s, sys: 19.2 ms, total: 4.39 s
Wall time: 4.37 s


### Calculating max High & min Low from start to end day

In [13]:
%%time
min_max = []
for i, row in data.iterrows():
    if 100*row.name.hour + row.name.minute == 900:
        min_max.append(0)
    else:
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        data_from_start_day = data[(data.index < current_time) & (data.index > current_date+' 08:30:00')]
        max_high = data_from_start_day['High'].max()
        min_low = data_from_start_day['Low'].min()
        if row['High'] >= max_high:
            min_max.append(1)
        elif row['Low'] <= min_low:
            min_max.append(-1)
        else:
            min_max.append(0)
data["min_max"] = min_max

CPU times: user 3.98 s, sys: 2.3 ms, total: 3.99 s
Wall time: 3.99 s


In [14]:
data

,Open,High,Low,Close,Volume,ma_line,above_ma,below_ma,total_above_ma_1w,total_below_ma_1w,...,min_OC,max_OC,upper_wick,tail,oc_dif,body,color,candlestick,model,min_max
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-06 09:00:00,1277.4,1277.4,1276.1,1276.6,5242,NaN,0,0,NaN,NaN,...,1276.6,1277.4,0.0,0.5,-0.8,0.8,black,,-,0
2024-03-06 09:05:00,1276.7,1276.7,1275.3,1275.6,2250,NaN,0,0,NaN,NaN,...,1275.6,1276.7,0.0,0.3,-1.1,1.1,black,,-,-1
2024-03-06 09:10:00,1275.6,1276.2,1275.5,1275.9,1772,NaN,0,0,NaN,NaN,...,1275.6,1275.9,0.3,0.1,0.3,0.3,white,,-,0
2024-03-06 09:15:00,1275.8,1276.7,1275.8,1276.5,3100,NaN,0,0,NaN,NaN,...,1275.8,1276.5,0.2,0.0,0.7,0.7,white,,-,0
2024-03-06 09:20:00,1276.6,1278.4,1276.5,1277.8,4666,NaN,0,0,NaN,NaN,...,1276.6,1277.8,0.6,0.1,1.2,1.2,white,,-,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-06 14:15:00,1310.1,1311.0,1309.1,1310.0,5772,1308.930,1,0,128.0,122.0,...,1310.0,1310.1,0.9,0.9,-0.1,0.1,black,,,0
2024-09-06 14:20:00,1310.4,1310.8,1309.7,1309.8,4554,1309.025,1,0,129.0,121.0,...,1309.8,1310.4,0.4,0.1,-0.6,0.6,black,,,0
2024-09-06 14:25:00,1309.9,1310.5,1308.8,1308.8,7565,1309.065,0,1,129.0,121.0,...,1308.8,1309.9,0.6,0.0,-1.1,1.1,black,,,0


In [16]:
data[(data.index > '2024-09-06 08:55:00') & (data.min_max != 0)]

,Open,High,Low,Close,Volume,ma_line,above_ma,below_ma,total_above_ma_1w,total_below_ma_1w,...,min_OC,max_OC,upper_wick,tail,oc_dif,body,color,candlestick,model,min_max
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-06 09:15:00,1307.5,1308.4,1307.1,1308.4,2128,1310.515,0,1,129.0,121.0,...,1307.5,1308.4,0.0,0.4,0.9,0.9,white,,bullish_engulfing,-1
2024-09-06 09:25:00,1307.9,1308.0,1305.6,1306.5,3659,1309.910,0,1,127.0,123.0,...,1306.5,1307.9,0.1,0.9,-1.4,1.4,black,,,-1
2024-09-06 09:45:00,1308.3,1309.7,1308.3,1309.6,6865,1308.725,1,0,126.0,124.0,...,1308.3,1309.6,0.1,0.0,1.3,1.3,white,,,1
2024-09-06 09:50:00,1309.7,1310.2,1309.5,1310.1,5317,1308.560,1,0,127.0,123.0,...,1309.7,1310.1,0.1,0.2,0.4,0.4,white,,,1
2024-09-06 09:55:00,1310.0,1310.2,1309.5,1309.8,2700,1308.505,1,0,128.0,122.0,...,1309.8,1310.0,0.2,0.3,-0.2,0.2,black,,bearish_tweezers_top,1
2024-09-06 10:00:00,1309.9,1310.5,1309.8,1310.5,3342,1308.435,1,0,129.0,121.0,...,1309.9,1310.5,0.0,0.1,0.6,0.6,white,,bullish_tasuki_line,1
2024-09-06 10:05:00,1310.4,1310.6,1309.7,1310.0,2045,1308.320,1,0,130.0,120.0,...,1310.0,1310.4,0.2,0.3,-0.4,0.4,black,,,1
2024-09-06 13:40:00,1310.0,1310.9,1309.6,1309.6,5878,1308.160,1,0,128.0,122.0,...,1309.6,1310.0,0.9,0.0,-0.4,0.4,black,,,1
2024-09-06 14:00:00,1310.0,1311.3,1309.8,1310.6,5202,1308.540,1,0,128.0,122.0,...,1310.0,1310.6,0.7,0.2,0.6,0.6,white,,,1
